In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 

from energy_consumption.feature_selection.extract.extract_SARIMAX_features import get_energy_and_SARIMAX_features

In [2]:
energydata = get_energy_and_SARIMAX_features()
energydata

100%|██████████| 4/4 [00:02<00:00,  1.43it/s]
C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\energy_consumption\feature_selection\collect\production_index.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energydata['half'][energydata.index.month.isin([7, 8, 9, 10, 11, 12])] = 1


,sun_hours,energy_consumption,saturday,working_day,holiday,index
date_time,,,,,,
2023-12-04 00:00:00,8.325918,47.03250,0,1,0,81.275
2023-12-04 01:00:00,8.325918,45.94450,0,1,0,81.275
2023-12-04 02:00:00,8.325918,45.03000,0,1,0,81.275
2023-12-04 03:00:00,8.325918,45.06575,0,1,0,81.275
2023-12-04 04:00:00,8.325918,46.43050,0,1,0,81.275
...,...,...,...,...,...,...
2023-12-27 08:00:00,8.094635,48.69925,0,1,0,81.275
2023-12-27 09:00:00,8.094635,50.96950,0,1,0,81.275
2023-12-27 10:00:00,8.094635,52.04400,0,1,0,81.275


In [21]:
model_real = sm.tsa.SARIMAX(energydata['energy_consumption'], exog=energydata[[
    'sun_hours','saturday', 'working_day', 'holiday','index']], 
    order=(1, 1, 1), seasonal_order=(1, 1, 1, 24), approximate_diffuse=True)
results_real = model_real.fit()

# forecasts
forecast_real = results_real.get_forecast(
    steps=100, exog=energydata[['sun_hours','saturday', 'working_day', 'holiday','index']][-100:])

pr_mean_real = forecast_real.predicted_mean
pr_mean_real

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['approximate_diffuse']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Maria\Documents\Setups\Lib\site-packag

2023-12-27 13:00:00    52.611929
2023-12-27 14:00:00    51.306353
2023-12-27 15:00:00    51.066561
2023-12-27 16:00:00    52.601958
2023-12-27 17:00:00    54.214422
                         ...    
2023-12-31 12:00:00    55.309215
2023-12-31 13:00:00    53.926451
2023-12-31 14:00:00    52.577214
2023-12-31 15:00:00    52.260136
2023-12-31 16:00:00    53.533532
Freq: H, Name: predicted_mean, Length: 100, dtype: float64

In [20]:
# do the same with the last 100 observations 
energy_actual_train = energydata.iloc[:-100]['energy_consumption']
exog_actual_train = energydata.iloc[:-100][[
    'sun_hours', 'saturday', 'working_day', 'holiday', 'index']]
model_st = sm.tsa.SARIMAX(energy_actual_train, exog=exog_actual_train,
    order=(1, 1, 1), seasonal_order=(1, 1, 1, 24), approximate_diffuse=True)
results_st = model_st.fit()

# forecasts
forecast_st = results_st.get_forecast(
    steps=100, exog=energydata[['sun_hours', 'saturday', 'working_day', 'holiday', 'index']].iloc[-200:-100,:])

predicted_mean = forecast_st.predicted_mean
actual_mean = energydata.iloc[-100:, :]['energy_consumption']

together = pd.DataFrame({'forecast': predicted_mean, 'actual': actual_mean})

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['approximate_diffuse']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['approximate_diffuse']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(ms

In [23]:
together['residuals'] = abs(together['actual'] - together['forecast'])
residuals = np.array(together['residuals'])

In [28]:
forecasts = pd.DataFrame({'mean_forecast': pr_mean_real, 'residuals': residuals})
forecasts

,mean_forecast,residuals
2023-12-27 13:00:00,52.611929,0.393317
2023-12-27 14:00:00,51.306353,3.113816
2023-12-27 15:00:00,51.066561,3.438026
2023-12-27 16:00:00,52.601958,4.269343
2023-12-27 17:00:00,54.214422,4.217293
...,...,...
2023-12-31 12:00:00,55.309215,5.836686
2023-12-31 13:00:00,53.926451,6.637058
2023-12-31 14:00:00,52.577214,6.890840
2023-12-31 15:00:00,52.260136,7.969579


In [26]:
from scipy.stats import norm

In [29]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

for q in quantiles: 
    forecasts[f'q{q}'] = forecasts['mean_forecast'] + forecasts['residuals']*norm.ppf(q, loc=0)


In [30]:
forecasts

,mean_forecast,residuals,q0.025,q0.25,q0.5,q0.75,q0.975
2023-12-27 13:00:00,52.611929,0.393317,51.841041,52.346640,52.611929,52.877217,53.382817
2023-12-27 14:00:00,51.306353,3.113816,45.203387,49.206117,51.306353,53.406590,57.409320
2023-12-27 15:00:00,51.066561,3.438026,44.328153,48.747647,51.066561,53.385474,57.804968
2023-12-27 16:00:00,52.601958,4.269343,44.234199,49.722330,52.601958,55.481586,60.969717
2023-12-27 17:00:00,54.214422,4.217293,45.948679,51.369901,54.214422,57.058944,62.480165
...,...,...,...,...,...,...,...
2023-12-31 12:00:00,55.309215,5.836686,43.869520,51.372430,55.309215,59.246000,66.748909
2023-12-31 13:00:00,53.926451,6.637058,40.918056,49.449824,53.926451,58.403079,66.934847
2023-12-31 14:00:00,52.577214,6.890840,39.071416,47.929413,52.577214,57.225015,66.083013
2023-12-31 15:00:00,52.260136,7.969579,36.640049,46.884737,52.260136,57.635535,67.880223


In [10]:
confidence_intervals_005 = forecast.conf_int()
confidence_intervals_050 = forecast.conf_int(alpha=0.5)

confidence_intervals_005.index.name = 'date_time'
confidence_intervals_005 = confidence_intervals_005.rename(
    columns={'lower energy_consumption':'q0.025', 
             'upper energy_consumption': 'q0.975'})

confidence_intervals_050.index.name = 'date_time'
confidence_intervals_050 = confidence_intervals_050.rename(
    columns={'lower energy_consumption':'q0.25', 
             'upper energy_consumption': 'q0.75'})

forecasts_merged = pd.merge(confidence_intervals_005, confidence_intervals_050, left_index=True, right_index=True)
forecasts_merged['q0.5'] = pr_mean

forecasts_merged = forecasts_merged.reindex(columns=['q0.025','q0.25', 'q0.5', 'q0.75', 'q0.975'])
forecasts_merged

# Extract rows for specific weekdays and hours
friday_12 = forecasts_merged[(forecasts_merged.index.weekday == 4) & (
    forecasts_merged.index.hour == 12)]
friday_16 = forecasts_merged[(forecasts_merged.index.weekday == 4) & (
    forecasts_merged.index.hour == 16)]
friday_20 = forecasts_merged[(forecasts_merged.index.weekday == 4) & (
    forecasts_merged.index.hour == 20)]

saturday_12 = forecasts_merged[(forecasts_merged.index.weekday == 5) & (
    forecasts_merged.index.hour == 12)]
saturday_16 = forecasts_merged[(forecasts_merged.index.weekday == 5) & (
    forecasts_merged.index.hour == 16)]
saturday_20 = forecasts_merged[(forecasts_merged.index.weekday == 5) & (
    forecasts_merged.index.hour == 20)]

# Create a common DataFrame to store the results
forecasts_submission = pd.concat(
    [friday_12, friday_16, friday_20, saturday_12, saturday_16, saturday_20])
forecasts_submission

,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,
2023-12-29 12:00:00,27.175434,45.813416,55.592786,65.372157,84.010139
2023-12-29 16:00:00,23.335767,43.359250,53.865594,64.371938,84.395422
2023-12-29 20:00:00,18.060352,39.473728,50.709350,61.944972,83.358347
2023-12-30 12:00:00,15.474501,41.732996,55.510858,69.288719,95.547214
2023-12-30 16:00:00,11.753922,39.296068,53.747462,68.198857,95.741002
2023-12-30 20:00:00,6.579622,35.409442,50.536481,65.663519,94.493340


In [11]:
from datetime import datetime
date_st = (datetime.today().strftime('%Y-%m-%d'))
hours = ['36 hour', '40 hour', '44 hour',
            '60 hour', '64 hour', '68 hour']

forecasts_submission.insert(0,'forecast_date',date_st)
forecasts_submission.insert(1,'target', 'energy')
forecasts_submission.insert(2, 'hours',hours)
forecasts_submission

,forecast_date,target,hours,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2023-12-29 12:00:00,2023-12-31,energy,36 hour,27.175434,45.813416,55.592786,65.372157,84.010139
2023-12-29 16:00:00,2023-12-31,energy,40 hour,23.335767,43.359250,53.865594,64.371938,84.395422
2023-12-29 20:00:00,2023-12-31,energy,44 hour,18.060352,39.473728,50.709350,61.944972,83.358347
2023-12-30 12:00:00,2023-12-31,energy,60 hour,15.474501,41.732996,55.510858,69.288719,95.547214
2023-12-30 16:00:00,2023-12-31,energy,64 hour,11.753922,39.296068,53.747462,68.198857,95.741002
2023-12-30 20:00:00,2023-12-31,energy,68 hour,6.579622,35.409442,50.536481,65.663519,94.493340


In [ ]:

def get_SARIMAX_forecasts(energydata=pd.DataFrame(), indexes=[47, 51, 55, 71, 75, 79],
                          quantiles=[0.025, 0.25, 0.5, 0.75, 0.975], periods=100):

    if energydata.empty:
        energydata = get_energy_and_SARIMAX_features()